In [1]:
import sys
import os
import pyodbc
from sklearn import ensemble, model_selection, preprocessing, tree
from sshtunnel import SSHTunnelForwarder
import pandas as pd

In [2]:
## SSH Tunnel for DB connection

server = SSHTunnelForwarder(
    (os.environ.get('DB_IP'), 22),
    ssh_username=os.environ.get('SSH_USER'),
    ssh_password=os.environ.get('SSH_PASS'),
    ssh_pkey=os.environ.get('SSH_PKEY'),
    remote_bind_address=(os.environ.get('DB_IP'), 5432),
    local_bind_address=("0.0.0.0", 10122)
)

server.start()

In [10]:
#conn = pyodbc.connect("Driver={PostgreSQL Driver};Server=127.0.0.1;Port=10122;Database=hertham;Uid=%s;Pwd=%s;" % (os.environ.get('DB_USER'), os.environ.get('DB_PASS')))
print(os.environ.get('DB_USER'))


None


In [15]:
%%time
df = pd.io.sql.read_sql("SELECT * FROM btc_usds WHERE \"createdAt\" > '2019-05-31 00:00:00.00+00' ORDER BY \"createdAt\" ASC", conn)
df = df.sort_values(['createdAt'])

CPU times: user 5.02 s, sys: 1.58 s, total: 6.6 s
Wall time: 10 s


In [19]:
df = (
    df
        .dropna(thresh=1)
        .assign(buy = lambda x: (x.shift(-1).close > x.close).astype(int))
)


In [20]:
# ignore = {'buy', 'closed', 'color', 'product', 'updatedAt', 'createdAt', 'timestamp', 'id', 'regression_10_r2', 'regression_20_r2'}
ignore = {'buy', 'closed', 'color', 'product', 'updatedAt', 'createdAt', 'timestamp', 'id'}
cols = [c for c in df.columns if c not in ignore]
X = df[cols]
y = df.buy
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=.3)


In [21]:
rf1 = ensemble.RandomForestClassifier()
rf1.fit(X_train, y_train)
rf1.score(X_test, y_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').